In [1]:
import pandas as pd
import numpy as np 

In [2]:
# Importuojame funkcijas
from functions import white_space
from functions import paskutinis_simbolis

In [3]:
# Perskaitome duomenis 
path = r'C:\Users\tvale\OneDrive - Girteka Logistics, UAB\Documents\02_mokymasis\final_poj\data\autobilis_data\autobilisspider\autobilisspider\autobilis.csv'
autobilis_df = pd.read_csv(path)

In [4]:
# Istriname stulpelius kurių nenaudosime tolimesniuose skaiciavimuose
autobilis_df = autobilis_df.drop(
    [
    'url', 
    'id',
    'vardas', 
    'miestas', 
    'title',
    'pardavejas',
    'telefonas',
    'vin'
    ]
    , axis=1
)

In [5]:
# Kainos stulpelis 
autobilis_df['kaina'] = white_space(autobilis_df['kaina']).str.replace('€', '').astype(int)

In [6]:
# Pagaminimo data. Eliminuojame \n
autobilis_df['pagaminimo_data'] = autobilis_df['pagaminimo_data'].replace(r'\n', '', regex=True)

# Eliminuojame jei nera iraso
autobilis_df = autobilis_df[autobilis_df['pagaminimo_data'].notna()]

# Imame pirmus 4 simbolius 
autobilis_df['pagaminimo_data'] = autobilis_df['pagaminimo_data'].str[:4].astype(int)

In [7]:
# Bukle. Nei reiksme yra nan pakeiciame i 'Be defektų'. 
# Paliekame tik tas eilutes kuriose yrašas 'Be defektų'
autobilis_df['bukle'] = np.where(
    (autobilis_df['bukle'].isna()), 'Be defektų', autobilis_df['bukle']
)

autobilis_df = autobilis_df.loc[autobilis_df['bukle'] == 'Be defektų']

In [8]:
# Vairo padėtis. Jei nėra įrašo pakeičiame į 'Kairėje'. 
# Visus įrašus kur yra 'Desineje' eliminuojame

autobilis_df['vairo_padetis'] = np.where(
    (autobilis_df['vairo_padetis'].isna()), 'Kairėje', autobilis_df['vairo_padetis']
)

autobilis_df = autobilis_df.loc[autobilis_df['vairo_padetis'] == 'Kairėje']

In [9]:
# Modelis. Jei nera reiksmes tokias eilutes eliminuoti 
autobilis_df = autobilis_df.loc[autobilis_df['modelis'].notna()]

In [10]:
# Galia stulpeli padaliname i dvi dalis ir paliekame tik skaitines reiksmes 
# Pirma dalis - galia_kw
# Antra dalis - galia_ag
autobilis_df[['galia_kw', 'galia_ag']] = autobilis_df['galia'].str.split('(', expand=True)

autobilis_df['galia_kw'] = white_space(autobilis_df['galia_kw']).str.replace('kW', '')
autobilis_df['galia_ag'] = white_space(autobilis_df['galia_ag']).str.replace('AG)', '')

In [11]:
# Rida. Paliekame tik skaitines reiksmes.
autobilis_df['rida'] = autobilis_df['rida'].str.replace(r'\n', '', regex=True)
autobilis_df['rida'] = white_space(autobilis_df['rida']).str.replace('km', '')

In [12]:
# Istriname nereikalingus stulpelius 
autobilis_df = autobilis_df.drop(
    [
        'galia',
        'vairo_padetis',
        'bukle',
        'varantieji_ratai'
    ], axis=1
)

In [13]:
autobilis_df = autobilis_df.reset_index(drop=True)

In [14]:
autobilis_df = autobilis_df.rename(
    columns={
        'marke': 'gamintojas',
        'pagaminimo_data': 'metai',
        'darbinis_turis': 'variklio_turis',
        'duru_skaicius': 'duru_sk'
    }
)

In [15]:
autobilis_df

,kaina,gamintojas,modelis,metai,variklio_turis,pavaru_deze,kuras,kebulas,rida,duru_sk,galia_kw,galia_ag
0,6850,Volvo,S60,2010,2.0,Mechaninė,Dyzelinas,Sedanas,336000,4/5,120,163
1,1600,Dodge,Durango,1999,5.9,Automatinė,Benzinas / dujos,Visureigis,NaN,4/5,184,250
2,950,Opel,Vectra,2006,2.2,Mechaninė,Benzinas,Hečbekas,NaN,4/5,114,155
3,1000,Audi,A6,2000,2.4,Automatinė,Benzinas / dujos,Sedanas,NaN,4/5,121,165
4,2900,BMW,Active Hybrid 7,2011,4.4,Automatinė,Benzinas / elektra,Sedanas,185000,4/5,339,461
...,...,...,...,...,...,...,...,...,...,...,...,...
2947,6599,Volkswagen,Polo,2012,1.6,Automatinė,Dyzelinas,Hečbekas,224000,4/5,66,90
2948,8200,Mercedes Benz,CLK320,2005,3.2,Automatinė,Benzinas,Kabrioletas/Roadster,266700,2/3,160,218
2949,3300,Mercedes Benz,270,2003,2.7,Automatinė,Dyzelinas,Kupė/Sportas,314017,2/3,125,170
2950,22450,Skoda,Octavia,2020,2.0,Automatinė,Benzinas,Universalas,57850,4/5,180,245
